<a href="https://colab.research.google.com/github/andrew-veriga/Tensorflow-labs/blob/master/PrepareFeaturesLabels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Setup

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

подключаем знакомые нам `tensorflow`, `numpy` и библиотеку рисования графиков 

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.2.0-rc2


используем здесь класс `tf.data.Dataset`, и чтобы создать себе некоторые данные, сделаем диапазон из 10 последовательных чисел. Будет удобно контролировать правильность наших действий, зная, что эти ряды должны равномерно возрастать. В реальных рядах такого удобства не будет, поэтому лучше заранее всё отладить на понятных числах.

In [4]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
   print(val.numpy())


0
1
2
3
4
5
6
7
8
9


<RangeDataset shapes: (), types: tf.int64>

Итак, теперь сделаем это немного интереснее. Мы будем использовать `dataset.window`, чтобы расширить наш набор данных с помощью сдвигающегося окна. Первый параметр `size` указывает размер окна, параметр `stride` указывает шаг, с которым отбираются значения, а `shift` - величину сдвига окна. Чтобы хорошо понять, как всё это работает, поэкспериментируйте с этими параметрами функции `window` поменяйте их в разных сочетаниях и посмотрите, что получится.\
*один фрагмент, выделенный окном в этом примере, выводится как горизонтальный ряд чисел, весь полученный набор должен выглядеть как стопка горизонтальных массивов*

In [34]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, stride=1,shift=1)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

0 2 4 6 8 
1 3 5 7 9 
2 4 6 8 
3 5 7 9 
4 6 8 
5 7 9 
6 8 
7 9 
8 
9 


Мы получили выходные данные первых пяти элементов, а затем вторых пяти элементов, а затем третьих пяти элементов и т. д. В конце набора данных, когда данных недостаточно, чтобы дать нам пять элементов, вы увидите более короткие строки. Но это не то, что нужно, ведь все признак-векторы должны быть одинаковой длины.\
Чтобы отбросить фрагменты с недостаточным количеством значений, мы установим параметр `drop_reminder` в значение true.


In [36]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


Теперь давайте поместим эти данные в списки, чтобы можно было начать использовать их в машинном обучении.\
`TensorFlow` любит, чтобы его данные были в формате `numpy`.\
 Ну, мы можем легко преобразовать его, вызвав метод `.numpy()`, и когда распечатаем его, увидим, что он теперь указан в квадратных скобках.

In [57]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda w: w.batch(5))
for w in dataset:
  print(w.numpy())


[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


Хорошо, теперь нам надо разделить данные на признак-векторы и метки. В каждом элементе надо для признак-вектора оставить все значения, кроме последнего, которое будет меткой. И это может быть достигнуто с помощью функции `map()`

In [59]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
# мы разбиваем каждый массив на две части: 
# первая содержит все элементы кроме последнего: [:-1], а вторая содержит только последний элемент: [-1:]
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
for x,y in dataset:
  print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


и теперь наш набор данных выглядит как хороший набор признаков и меток.\
И осталось только перемешать его. Это поможет нам избежать предвзятости в  последовательности. \
И это возможно с помощью метода `.shuffle()`. Мы вызовем его с параметром `buffer_size=10`, потому что это то максимальное количество строк, которые у нас могут быть.\
Многократные прогоны покажут строки в разной последовательности, потому что они перемешиваются случайным образом.

In [69]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda w: w.batch(5))
dataset = dataset.map(lambda w: (w[:-1], w[-1:]))
dataset = dataset.shuffle(buffer_size=10)
for x,y in dataset:
  print(x.numpy(), y.numpy())

[1 2 3 4] [5]
[5 6 7 8] [9]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[0 1 2 3] [4]


Наконец, пакетирование данных. и это делается с помощью метода `.batch()'.\
Устанавливая размер пакета в 2, мы объединим наши данные  по два x и два y вместе. Ранее мы не пользовались таким разбиением данных на пакеты, keras все делал за нас, если это было нужно. Но теперь попробуем другой подход

In [73]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda w: w.batch(5))
dataset = dataset.map(lambda w: (w[:-1], w[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())


x =  [[2 3 4 5]
 [1 2 3 4]]
y =  [[6]
 [5]]
x =  [[0 1 2 3]
 [4 5 6 7]]
y =  [[4]
 [8]]
x =  [[5 6 7 8]
 [3 4 5 6]]
y =  [[9]
 [7]]


Итак, это рабочая книга с кодом, который разбивает ряд данных на окна. Попробуйте поиграть с данными, сделайте в последней ячейке изменения параметров в тех методах, которые вам показались непонятными и как только вы ознакомитесь с тем, что он делает, переходите к следующей главе.
Там вы увидите сезонный набор данных, и с помощью этой техники сдвигающегося окна вы увидите, как получить из него x и y, которые можно подавать в нейронную сеть и посмотреть, как она работает с прогнозированием.